In [58]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

import pandas as pd
import numpy as np

import math
import time

import json

from util import get_thegraph_client, gq, gql, gql_all2, gql_all, iter_gql_all2, iter_gql_all, pd_read_dir, pd_read_json, load_from_mysql


In [59]:
# ENS.
ENS_SUBGRAPH = "EjtE3sBkYYAwr45BASiFp8cSZEvd1VHTzzYFvJwQUuJx"

ENS_HOSTED_SERVICE_SUBGRAPH = "QmNus9NhDGKCYixuKBBrJwoc1nFwjtevRkMGvWNn7HJHhE"

ENDPOINT = "https://gateway.thegraph.com/api/" + config['THEGRAPH_KEY'] + \
    "/subgraphs/id/" + ENS_SUBGRAPH

ENDPOINT = "https://api.thegraph.com/subgraphs/name/ensdomains/ens"

# https://thegraph.com/explorer/subgraphs/EjtE3sBkYYAwr45BASiFp8cSZEvd1VHTzzYFvJwQUuJx?view=Playground&chain=mainnet

In [60]:
snapshot_ids = pd_read_json("../data/selected_spaces.json")
snapshot_ids

,0
0,lido-snapshot.eth
1,curve.eth
2,aave.eth
3,uniswap
4,instadapp-gov.eth
...,...
205,xxtka.eth
206,klimadao.eth
207,halodao.eth
208,ecashxec.eth


In [61]:
thegraph = get_thegraph_client(ENDPOINT)

In [62]:
query = gql(
"""
query Domains {
  domains(first: 1, where: {name: "aave.eth"}) {
    id
    name
    labelName
    owner {
      id
      registrations {
        id
        registrant {
          id
        }
      }
    }
    subdomains {
      id
      name
      owner {
        id
      }
    }
    subdomainCount
    createdAt
    registration {
      events {
        registration {
          id
          domain {
            name
            owner {
              id
            }
            createdAt
            ttl
          }
        }
      }
      expiryDate
    }
    wrappedDomain {
      labelName
      owner {
        id
      }
      fuses
    }
  }
}
""")

In [63]:
result = await thegraph.execute_async(query)
print(result)

{'domains': [{'id': '0x6e6cb1c71165f8db6f8c358b209678a6fc9e938fb4182d16e7853b40c687b95c', 'name': 'aave.eth', 'labelName': 'aave', 'owner': {'id': '0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712', 'registrations': [{'id': '0xbbad42e9bd6278fe7e4e72d9f78e3ec5fe009984005bec5149437256c6df0416', 'registrant': {'id': '0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712'}}, {'id': '0xe0a39fef8aa5887c76606e7a14a5db8dc6676191e4866b2a609a3aa003b18329', 'registrant': {'id': '0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712'}}]}, 'subdomains': [{'id': '0x54156cdf68456f6cc7e51069ac1df89577dbcf935f52db34539cb41d7333b555', 'name': 'tokenlist.aave.eth', 'owner': {'id': '0x357b7e9acd156c0f930b75c6ae6a42f3d9173042'}}, {'id': '0x7a1f4982c9c9dcbc13072df859a8c7a6cee61dad7464892a05176a935457fe13', 'name': '[0a45e4e8674730759940f05a54e92cfd108c17cf6aa82041a0922e208db7b0f2].aave.eth', 'owner': {'id': '0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712'}}], 'subdomainCount': 2, 'createdAt': '1572233826', 'registration': {'events': [{'re

In [64]:
res = await gql_all2(query, 
               "domains",
               thegraph,
               limit=2)

1 1
2 2
**Limit reached:  2


In [65]:
res

[{'id': '0x6e6cb1c71165f8db6f8c358b209678a6fc9e938fb4182d16e7853b40c687b95c',
  'name': 'aave.eth',
  'labelName': 'aave',
  'owner': {'id': '0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712',
   'registrations': [{'id': '0xbbad42e9bd6278fe7e4e72d9f78e3ec5fe009984005bec5149437256c6df0416',
     'registrant': {'id': '0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712'}},
    {'id': '0xe0a39fef8aa5887c76606e7a14a5db8dc6676191e4866b2a609a3aa003b18329',
     'registrant': {'id': '0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712'}}]},
  'subdomains': [{'id': '0x54156cdf68456f6cc7e51069ac1df89577dbcf935f52db34539cb41d7333b555',
    'name': 'tokenlist.aave.eth',
    'owner': {'id': '0x357b7e9acd156c0f930b75c6ae6a42f3d9173042'}},
   {'id': '0x7a1f4982c9c9dcbc13072df859a8c7a6cee61dad7464892a05176a935457fe13',
    'name': '[0a45e4e8674730759940f05a54e92cfd108c17cf6aa82041a0922e208db7b0f2].aave.eth',
    'owner': {'id': '0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712'}}],
  'subdomainCount': 2,
  'createdAt': '157223382

In [66]:
## Utility function to save intermediate and final results.
def save_json3(out, data_dir, file_template, save_counter):

    # Pandas has problem load pure json saves.
    # Hence we create a pandas Dataframe and save it.
    # nonlocal append
    # flag = "a" if append else "w"
    # with open("data/" + save, flag) as f:
    #     json.dump(out, f)
    #     print("Saved.")

        
    sv = str(save_counter)
    sv = sv.zfill(5)
    save_counter += 1

    filename = file_template.replace('.json', '_' + sv + '.json')
    
    out_str = "Saved " + sv

    df = pd.DataFrame(out)
    df.to_json(data_dir + "/" + filename, orient="records")

    print(out_str)


import time
def do_throttle(throttle):
    ## Throttle.
    if throttle['max'] is not None:

        # print(throttle)

        ## Is it time to check?
        elapsed = time.time() - throttle['time']
        if elapsed >= throttle['sec']:

            # Do we have too many requests?
            if throttle['req'] >= throttle['max']:

                # Sleep for at most the one cycle of time-check.
                wait_time = min(0.3  * (throttle['req'] - throttle['max']), 
                                throttle['wait_max'])
                if wait_time < throttle['wait_min']: wait_time = throttle['wait_min']

                print("Throttling: " + str(throttle['req']) + " in " + str(elapsed))
                time.sleep(wait_time)
                
                # Restart counting.
                throttle['time'] = time.time()
                throttle['req'] =  0

        
        throttle['req'] += 1

async def gql_all2(query, 
                  field,
                  client, 
                  initial_list=None, 
                  counter = True, limit=None, 
                  save=None,
                  save_filename=None,
                  save_interval=10,
                  save_every=2000,
                  clear_on_save = False, 
                  append=True, 
                  rest=False, 
                  data_dir="data", 
                  save_counter = None, 
                  vars=None,
                  retry=False, 
                  max_retry=3, 
                  throttle={
                    "sec": 10,
                    "max": None,
                    "req": 0,
                    "time": None,
                    "sleep": 1
                  }):

    ## Result from server.
    res = None

    ## The returned value and the varible used to accumulate results.
    out = []

    def should_save(last=False):
        nonlocal save
        if not save: return False

        nonlocal save_every
        if save_every is not None:
            nonlocal out
            if last: return len(out) > 0
            return len(out) > save_every
        
        else:
            nonlocal my_counter, save_interval
            res = my_counter % save_interval == 0
            if last: res = not res
            return res
         
    ## Utility function to save intermediate and final results.
    def save_json():

        # Pandas has problem load pure json saves.
        # Hence we create a pandas Dataframe and save it.
        # nonlocal append
        # flag = "a" if append else "w"
        # with open("data/" + save, flag) as f:
        #     json.dump(out, f)
        #     print("Saved.")

        nonlocal out
        df = pd.DataFrame(out)

        # Introduced save_filename.
        filename = save_filename if save_filename is not None else save

        if clear_on_save:
            
            nonlocal save_counter
            
            sv = str(save_counter)
            sv = sv.zfill(5)
            save_counter += 1

            filename = filename.replace('.json', '_' + sv + '.json')
            
            out = []
            out_str = "Saved and cleared."
        else:
            # Removed when save_filename introduced.
            # filename = save
            out_str = "Saved."
        
        df.to_json(data_dir + "/" + filename, orient="records")
        print(out_str)


    def handleException(res):
        
        nonlocal n_retry

        exc_type, exc_value, exc_traceback = sys.exc_info()

        print("*** print_tb:")
        traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)
        # print("*** print_exception:")
        # traceback.print_exception(exc_value, limit=2, file=sys.stdout)
        print("*** print_exc:")
        traceback.print_exc(limit=2, file=sys.stdout)
        print("*** format_exc, first and last line:")
        formatted_lines = traceback.format_exc().splitlines()
        print(formatted_lines[0])
        print(formatted_lines[-1])
        # print("*** format_exception:")
        # print(repr(traceback.format_exception(exc_value)))
        print("*** extract_tb:")
        print(repr(traceback.extract_tb(exc_traceback)))
        print("*** format_tb:")
        print(repr(traceback.format_tb(exc_traceback)))
        print("*** tb_lineno:", exc_traceback.tb_lineno)
        
        print("")

        if res is not None:
            print("Last res")
            print(res)
        else:
            print("No res yet")

        print("")
        if (not retry) or (n_retry == 0):
            print("**An error occurred, exiting early.**")
            if save and len(out) > 0: save_json()
            
            return False
        else:
            if retry:
                n_retry -= 1
                print("**An error occurred, n_retry: " + str(n_retry) + " **")
                return True
            else:
                return False   

    ## Load initial list.
    ## If no skip is provided, then skip is set to the length of
    ## the initial list, otherwise we use the user-specified value
    if initial_list:
        out = initial_list
        skip = len(out)
    else:
        skip = 0

    first = 100

    ## Make a GQL query object, if necessary.
    if not rest and type(query) == str:
        query = gql(query)
        
    if save and (save_counter is None):
        save_counter = pd_read_dir(data_dir, count_only=True) + 1
        print("Save counter auto-detected:", save_counter)

    n_retry = max_retry
    my_counter = 0
    fetch = True

    # If throttle is enabled, we need to keep track of time.
    if ('max' in throttle) and (throttle['max'] is not None):
        throttle = _gql_all_set_default_throttle(throttle)


    while fetch:
        
        do_throttle(throttle)

        # Re-init variables.
        res = None
        thrown = False
        my_counter += 1
        
        if limit and my_counter > limit:
            print('**Limit reached: ', limit)
            fetch = False
            continue

        if rest:

            raise Exception("rest not supported for now")

            ## TODO: for tally it needs to be adjusted.

            # idx_bracket = query.find("{")
            # q = "query " + query[idx_bracket:]
            # # Building query manually.
            # # q = query.replace("($first: Int!, $skip: Int!)", "")
            # q = q.replace("$first", str(first))
            # q = q.replace("$skip", str(skip))
            # # print(q)

            # ## Optional additional variables.
            # if vars:
            #     for v in vars:
            #         # where = v + ": $" + v
            #         # replaced = v + ": " + str(vars[v])
            #         q = q.replace("$" + v, '"' + str(vars[v]) + '"')

            # # print(q)

            # try:
            #     if do_tally:
            #         res = tally_rest(q)
            #     else: 
            #         res = snapshot_rest(q)
                
            # except Exception as e:
            #      fetch = handleException(res)
            #      thrown = True
        else:
           
            try:
                res = await client.execute_async(query, variable_values=vars)
                    
            except Exception as e:
                fetch = handleException(res)
                thrown = True

        ## If no error was raised we save results and increment.
        ## If an error was raised, we will either exit early or repeat
        ## the same iteration.
        if not thrown:
            n_retry = max_retry
            if not res[field]:
                print('**I am done fetching!**')
                fetch = False
            else:
                out.extend(res[field])
                skip += first
                if counter: print(my_counter, len(out))

                if should_save(): save_json()

    if should_save(): save_json()

    return out

In [67]:
async def iter_gql_all2(iter, iter_var, opts, single=False):
    
    ## Temporary container, it will be dumped.
    res = []
    ## Kept and returned.
    empty_res = []

    ## Set defaults.

    if not 'data_dir' in opts:
        raise Exception("opts.data_dir is missing")

    if 'save_counter' in opts:
        save_counter = opts['save_counter']
    else:
        save_counter = pd_read_dir(opts['data_dir'], count_only=True) + 1
        print("save_counter", save_counter)

    if not 'vars' in opts: 
        opts['vars'] = {}

    opts['save'] = False

    if not 'save_filename' in opts:
        save_filename = iter_var + ".json"
    else:
        save_filename = opts['save_filename']

    throttle = opts['throttle']

    if single:
        # If throttle is enabled, we need to keep track of time.
        if ('max' in throttle) and (throttle['max'] is not None):
            throttle = _gql_all_set_default_throttle(throttle)

    for id in iter:
        
        # For Tally it was:
        # opts['vars'][iter_var] = [ id ]
        
        opts['vars'][iter_var] = id

        print(id)

        if single:

            do_throttle(throttle)
            
            one_res = await opts['client'].execute_async(opts['query'],
                                                         variable_values=opts['vars'])

            one_res = one_res[opts['field']]
            
            # try:
                # one_res = await opts.client.execute_async(**opts) 
            # except Exception as e:
            #     ## TODO: print e.
            #     one_res = []

        else:    
            one_res = await gql_all2(**opts)

        if len(one_res) > 0:
            res.extend(one_res)
        else:
            empty_res.extend([id])

        print(len(res))

        ## Save it...
        if len(res) > opts['save_every']:
            save_json3(res, opts['data_dir'], save_filename, save_counter)
            save_counter += 1
            res = []

    ## Last iter.
    if len(res) > 0:
        save_json3(res, opts['data_dir'], save_filename, save_counter)
        
    return empty_res

In [68]:
def _gql_all_set_default_throttle(throttle):
    # The first iteration is probably None
    if ('time' not in throttle) or (throttle['time'] is None):
        throttle['time'] = time.time()

    if ('req' not in throttle) or (throttle['req'] is None):
        throttle['req'] = 0

    if ('sec' not in throttle) or (throttle['sec'] is None):
        throttle['sec'] = 10

    if ('wait_max' not in throttle) or (throttle['wait_max'] is None):
        throttle['wait_max'] = throttle['sec'] + 2
    
    if ('wait_min' not in throttle) or (throttle['wait_min'] is None):
        throttle['wait_min'] = 1

    return throttle

In [69]:
data_dir = "../data/thegraph/ens/domains/"

throttle = {
    "sec": 8,
    "max": 26,
    "wait_max": 12,
    "wait_min": 3
}

# Proposals.
field = "domains"
query = gq("thegraph_ens", True)

opts = {
    
    # Main options.

    "query": query,
    "field": field, 
    "client": thegraph,

    # Saving options.

    "data_dir": data_dir,
    "save_every": 250,

    # Throttling and error handling options.

    "retry":  False,    # Retry on error
    "throttle":  throttle
}


ids = list(snapshot_ids[0])

empties = await iter_gql_all2(ids,
                          "name",
                          opts,
                          single=True
                          )

save_counter 3
lido-snapshot.eth
1
curve.eth
2
aave.eth
3
uniswap
3
instadapp-gov.eth
4
comp-vote.eth
5
rocketpool-dao.eth
6
frax.eth
7
balancer.eth
8
gauges.aurafinance.eth
9
ybaby.eth
10
dydxgov.eth
11
sushigov.eth
12
synthetix-stakers-poll.eth
13
eulerdao.eth
14
morpho.eth
15
olympusdao.eth
16
abracadabrabymerlinthemagician.eth
17
community.nexusmutual.eth
18
pancake
18
snapshot.dcl.eth
19
cakevote.eth
20
huwacoin.eth
21
index-coop.eth
22
apollodao.eth
23
bancornetwork.eth
24
snxgov.eth
Throttling: 26 in 12.941565036773682
25
dcip.eth
26
ethip.eth
27
polygondaoo.eth
28
dapang.eth
29
aavegotchi.eth
30
dapang1.eth
31
aggris88.eth
32
cvx.eth
33
goodmorningnews.eth
34
yokaiswap.eth
35
fabien.eth
36
jbdao.eth
37
17707.eth
38
狂奔的蜗牛1.eth
39
primerating.eth
40
sharkdao.eth
41
狂奔的蜗牛2.eth
42
xdaistake.eth
43
karastar.eth
44
bgansv2.eth
45
frami.eth
46
yam.eth
47
rallygov.eth
48
levidao.eth
49
qidao.eth
50
zhulindong.eth
Throttling: 26 in 13.15499472618103
51
zhudapang3.eth
52
dorg.eth
53
hbot

In [70]:
ens_data = pd_read_dir("../data/thegraph/ens/domains/")

In [71]:
empties = [item + ".eth" for item in empties]

In [72]:
empties[5] = 'genesis-dao.hopr.eth'

In [73]:
empties2 = await iter_gql_all2(empties,
                          "name",
                          opts,
                          single=True
                          )

save_counter 4
uniswap.eth
1
pancake.eth
2
acryptos.eth
3
mantra-dao.eth
4
genesis-dao.hopr.eth.eth
4
genesis-dao.hopr.eth
4
layer2dao.org.eth
4
Saved 00004


In [74]:
ens_data.iloc[0]

id                0x9273297cad24984fb18f583943154a9feba9a7302557...
name                                              lido-snapshot.eth
labelName                                             lido-snapshot
owner             {'id': '0xc8381ca290c198f5ab739a1841ce8aedb0b3...
subdomains                                                       []
subdomainCount                                                    0
createdAt                                                1611991059
registration      {'events': [{'registration': {'id': '0xe155ac5...
wrappedDomain                                                  None
Name: 0, dtype: object

In [108]:
ens_addresses = pd.DataFrame()
cols = ['id', 'name', 'subdomain_id', 'subdomain', 'address', 'type']

def extract_addresses(ens_data):
    global ens_addresses

    cleaned_data = [
        [ ens_data['id'], ens_data['name'], '', '', ens_data['owner']['id'], 'owner' ]
    ]
    
    print(ens_data['name'], len(ens_addresses))

    ## Keep track of all addresses found.
    all_addresses = {  ens_data['owner']['id']: True }

    ## Try to extract past registrant addresses from Events.
    if (ens_data['registration'] is not None) and \
        (ens_data['registration']['events'] is not None):

        events = ens_data['registration']['events']
        
        for i in range(len(events)):
            
            new_address = events[i]['registration']['domain']['owner']['id']

            if not new_address in all_addresses:
                cleaned_data.extend([
                     [ 
                        ens_data['id'], ens_data['name'], '',
                        '', new_address, 'past_owner' 
                     ]
                ])
                all_addresses[new_address] = True

    ## Try to extract past registrant addresses from Events.
    if ens_data['subdomains'] is not None:
        subdomains = ens_data['subdomains']
        
        for i in range(len(subdomains)):

            new_address = subdomains[i]['owner']['id']

            if not new_address in all_addresses:
                cleaned_data.extend([
                    [ 
                        ens_data['id'], ens_data['name'], subdomains[i]['id'], 
                        subdomains[i]['name'], new_address, 'subdomain_owner' 
                    ]
                ])
                all_addresses[new_address] = True

    
    ens_df = pd.DataFrame.from_records(cleaned_data, columns=cols)
    
    ens_addresses = pd.concat([ens_addresses, ens_df])
    

ens_data.apply(extract_addresses, axis=1)

lido-snapshot.eth 0
{'0xc8381ca290c198f5ab739a1841ce8aedb0b330d5': True}
curve.eth 1
{'0x312c44739556797a4589e0be0084159c74ccaa6a': True}
aave.eth 2
{'0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712': True, '0x357b7e9acd156c0f930b75c6ae6a42f3d9173042': True}
instadapp-gov.eth 4
{'0x2888eb67ac7ee9272fafd9277e60bd851b15e1b1': True}
comp-vote.eth 5
{'0x2b384212edc04ae8bb41738d05ba20e33277bf33': True}
rocketpool-dao.eth 6
{'0xd2a4848a6644749e652c1d9398b5aa317f57395b': True}
frax.eth 7
{'0x8da390541b36076d22b656d86b7dffc6966d4f94': True}
balancer.eth 8
{'0x10a19e7ee7d7f8a52822f6817de8ea18204f2e4f': True}
gauges.aurafinance.eth 9
{'0xfee0bbe31345a7c27368534fef45a57133ff3a86': True}
ybaby.eth 10
{'0x7a1057e6e9093da9c1d4c1d049609b6889fc4c67': True}
dydxgov.eth 11
{'0x410210ef65f69e1ac515974604323794fab6ae1c': True}
sushigov.eth 12
{'0x19b3eb3af5d93b77a5619b047de0eed7115a19e7': True}
synthetix-stakers-poll.eth 13
{'0x1a7fc76f1ac7fecb71256a79482dd5ad879f293a': True}
eulerdao.eth 14
{'0x8da6a996d8f0cf

0      None
1      None
2      None
3      None
4      None
       ... 
198    None
199    None
200    None
201    None
202    None
Length: 410, dtype: object

In [106]:
ens_addresses

,id,name,subdomain_id,subdomain,address,type
0,0x9273297cad24984fb18f583943154a9feba9a7302557...,lido-snapshot.eth,,,0xc8381ca290c198f5ab739a1841ce8aedb0b330d5,owner
0,0x61964dc11f819a1253730272af9a9f035b6861ad888a...,curve.eth,,,0x312c44739556797a4589e0be0084159c74ccaa6a,owner
0,0x6e6cb1c71165f8db6f8c358b209678a6fc9e938fb418...,aave.eth,,,0x3a44eb3f751ec2f737f9f9b0b11b5984e8f3d712,owner
1,0x6e6cb1c71165f8db6f8c358b209678a6fc9e938fb418...,aave.eth,0x54156cdf68456f6cc7e51069ac1df89577dbcf935f52...,tokenlist.aave.eth,0x357b7e9acd156c0f930b75c6ae6a42f3d9173042,subdomain_owner
0,0x8da5c2e8c917584f9e8214a1129fcb4f7f38dee13b99...,instadapp-gov.eth,,,0x2888eb67ac7ee9272fafd9277e60bd851b15e1b1,owner
...,...,...,...,...,...,...
0,0xe5f096a9b3a7b31b4a55660cc5771f6d79e15738e46f...,xxtka.eth,,,0xe82f83566480c5273404d86ea1c447e6d57d307b,owner
0,0xa286855008c78d7af5598a64c2eb524f6c913ad96c82...,klimadao.eth,,,0x693ad12dba5f6e07de86faa21098b691f60a1bea,owner
0,0xae20bbb1172e52f630d3157d2458de13ac9230db26ca...,halodao.eth,,,0x30ca8f1e13978a7a819af012665cfa7e49f1c554,owner
0,0x9c79730632d1ab2f4f4a21dd542af8f6c0d812bd111f...,ecashxec.eth,,,0x3aaa1b22b24dcfd0db68660c891e71915aa3dfe7,owner


In [109]:
ens_addresses['type'].value_counts()

owner              410
subdomain_owner    242
Name: type, dtype: int64

In [113]:
## Removing duplicates. Why are there duplicates??
ens_addresses_clean = ens_addresses.drop_duplicates(subset=["name", "subdomain", "address"])

In [114]:
ens_addresses_clean['type'].value_counts()

owner              207
subdomain_owner    122
Name: type, dtype: int64

In [116]:
ens_addresses_clean.to_json("../data/thegraph/ens/addresses.json", orient="records")

In [27]:
ens_addresses.iloc[100]['name']

In [49]:
ens_data.iloc[0]['registration']

{'events': [{'registration': {'id': '0xe155ac5c70f13b65ecff90a9f1359ed7d740e9d5ad5616a5c5ef47391fdc79c1',
    'domain': {'name': 'lido-snapshot.eth',
     'owner': {'id': '0xc8381ca290c198f5ab739a1841ce8aedb0b330d5'},
     'createdAt': '1611991059',
     'ttl': None}}},
  {'registration': {'id': '0xe155ac5c70f13b65ecff90a9f1359ed7d740e9d5ad5616a5c5ef47391fdc79c1',
    'domain': {'name': 'lido-snapshot.eth',
     'owner': {'id': '0xc8381ca290c198f5ab739a1841ce8aedb0b330d5'},
     'createdAt': '1611991059',
     'ttl': None}}}],
 'expiryDate': '1927560579'}

In [17]:
list(ids[0])

['lido-snapshot.eth',
 'curve.eth',
 'aave.eth',
 'uniswap',
 'instadapp-gov.eth',
 'comp-vote.eth',
 'rocketpool-dao.eth',
 'frax.eth',
 'balancer.eth',
 'gauges.aurafinance.eth',
 'ybaby.eth',
 'dydxgov.eth',
 'sushigov.eth',
 'synthetix-stakers-poll.eth',
 'eulerdao.eth',
 'morpho.eth',
 'olympusdao.eth',
 'abracadabrabymerlinthemagician.eth',
 'community.nexusmutual.eth',
 'pancake',
 'snapshot.dcl.eth',
 'cakevote.eth',
 'huwacoin.eth',
 'index-coop.eth',
 'apollodao.eth',
 'bancornetwork.eth',
 'snxgov.eth',
 'dcip.eth',
 'ethip.eth',
 'polygondaoo.eth',
 'dapang.eth',
 'aavegotchi.eth',
 'dapang1.eth',
 'aggris88.eth',
 'cvx.eth',
 'goodmorningnews.eth',
 'yokaiswap.eth',
 'fabien.eth',
 'jbdao.eth',
 '17707.eth',
 '狂奔的蜗牛1.eth',
 'primerating.eth',
 'sharkdao.eth',
 '狂奔的蜗牛2.eth',
 'xdaistake.eth',
 'karastar.eth',
 'bgansv2.eth',
 'frami.eth',
 'yam.eth',
 'rallygov.eth',
 'levidao.eth',
 'qidao.eth',
 'zhulindong.eth',
 'zhudapang3.eth',
 'dorg.eth',
 'hbot-prp.eth',
 'dunjia.e